<a href="https://colab.research.google.com/github/RaviBekka/Deep-Learning/blob/main/notebook7400dd827d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basics of Deep Learning:Classification

In this notebook I will be showing you a simple model that can be used for classification and also show the importance of model architecture by comparing it with a CNN based algorithm on Cifar-10 Dataset

# 1 Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


# 2 Load Dataset

In [ ]:
cifar10 = tf.keras.datasets.cifar10

# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# 3 Preprocessing

In [ ]:
# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# flatten the label values
y_train, y_test = y_train.flatten(), y_test.flatten()

In [ ]:
fig, ax = plt.subplots(5, 5)
k = 0

for i in range(5):
    for j in range(5):
        ax[i][j].imshow(x_train[k], aspect='auto')
        k += 1

plt.show()

# 4 Basic Model
# 4.1 Defining the model

In [ ]:
# Define the sequential model
model = tf.keras.Sequential([
    # Flattening the input
    tf.keras.layers.Flatten(input_shape=(32, 32, 3)),  # Input Layer

    # Hidden layer with 256 units and ReLU activation
    tf.keras.layers.Dense(256, activation='relu'),  # First Hidden Layer

    # Hidden layer with 128 units and ReLU activation
    tf.keras.layers.Dense(128, activation='relu'),  # Second Hidden Layer

    # Hidden layer with 64 units and ReLU activation
    tf.keras.layers.Dense(64, activation='relu'),  # Third Hidden Layer

    # Output layer with 10 units for class probabilities
    tf.keras.layers.Dense(10, activation='softmax')  # Output Layer
])


# 4.2 Compile the model

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

# 4.3 Training the model

In [ ]:
%%time
# Fit the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=25)

# 4.4 Evaluate the model

In [ ]:
# Plot accuracy per iteration
plt.plot(history.history['accuracy'], label='acc', color='red')
plt.plot(history.history['val_accuracy'], label='val_acc', color='green')
plt.legend()

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

# 5 CNN Based Architecture
# 5.1 Defining the Model

In [ ]:

# Define the number of classes
K = len(set(y_train))
print("Number of classes:", K)

# Build the model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))

# Hidden layers
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(K, activation='softmax'))

# Model summary
model.summary()


# 5.2 Compile the model

In [ ]:
# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 5.3 Define Callbacks

In [ ]:

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001)


# 5.4 Train the model

In [ ]:

%%time
# Fit
r = model.fit(
  x_train, y_train, validation_data=(x_test, y_test), epochs=50,
  callbacks=[early_stopping, reduce_lr])

# 5.5 Evaluating the model

In [ ]:
# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='acc', color='red')
plt.plot(r.history['val_accuracy'], label='val_acc', color='green')
plt.legend()

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')


# 5.6 Checking the final model with the some examples

In [ ]:
# label mapping

labels = '''airplane automobile bird cat deer dog frog horseship truck'''.split()

# select the image from our test dataset
image_number = 290

# display the image
plt.imshow(x_test[image_number])

# load the image in an array
n = np.array(x_test[image_number])

# reshape it
p = n.reshape(1, 32, 32, 3)

# pass in the network for prediction and
# save the predicted label
predicted_label = labels[model.predict(p).argmax()]

# load the original label
original_label = labels[y_test[image_number]]

# display the result
print("Original label is {} and predicted label is {}".format(
    original_label, predicted_label))